In [1]:
## my data is a panel
## (robust) clustered standard errors
## fixed effects or random effects
##data clean up DALY WASH
import pandas as pd
import numpy as np
import os

##set a working directory
user = os.path.expanduser('~')
user

os.makedirs(f'{user}/Desktop/', exist_ok=True) 

path = os.chdir(f'{user}/Desktop/'+'GBD wash/')
file_list = os.listdir(path)
file_list

['.DS_Store', 'IHME-GBD_2019_DATA-820325b5-1.csv']

In [2]:
# =============================================================================
## 1. IMPORT THE DATA
# =============================================================================
#import the csv files into one dataframe
#import the csv files into one dataframe
file= pd.concat(map(pd.read_csv, ['IHME-GBD_2019_DATA-820325b5-1.csv']))
df = file.copy()
display(df.head())

,measure_id,measure_name,location_id,location_name,sex_id,sex_name,age_id,age_name,cause_id,cause_name,rei_id,rei_name,metric_id,metric_name,year,val,upper,lower
0,2,DALYs (Disability-Adjusted Life Years),170,Congo,3,Both,22,All ages,956,Respiratory infections and tuberculosis,82,"Unsafe water, sanitation, and handwashing",3,Rate,1990,1094.230977,1838.052066,505.145782
1,2,DALYs (Disability-Adjusted Life Years),170,Congo,3,Both,22,All ages,956,Respiratory infections and tuberculosis,238,No access to handwashing facility,3,Rate,1990,1094.230977,1838.052066,505.145782
2,2,DALYs (Disability-Adjusted Life Years),182,Malawi,3,Both,22,All ages,956,Respiratory infections and tuberculosis,82,"Unsafe water, sanitation, and handwashing",3,Rate,1990,2577.721592,4112.196439,1112.851738
3,2,DALYs (Disability-Adjusted Life Years),182,Malawi,3,Both,22,All ages,956,Respiratory infections and tuberculosis,238,No access to handwashing facility,3,Rate,1990,2577.721592,4112.196439,1112.851738
4,2,DALYs (Disability-Adjusted Life Years),182,Malawi,3,Both,22,All ages,957,Enteric infections,82,"Unsafe water, sanitation, and handwashing",3,Rate,1990,17485.375252,22842.570838,12345.249184


In [3]:
list(df.location_name.unique())

['Congo',
 'Malawi',
 'Madagascar',
 'Equatorial Guinea',
 'Angola',
 'Comoros',
 'Ethiopia',
 'Eritrea',
 'Botswana',
 'Burundi',
 'Central African Republic',
 'Niger',
 'Djibouti',
 'Chad',
 'Mauritania',
 'Cabo Verde',
 'Mozambique',
 'Zambia',
 "Côte d'Ivoire",
 'Gabon',
 'Eswatini',
 'Zimbabwe',
 'Benin',
 'Democratic Republic of the Congo',
 'Somalia',
 'Lesotho',
 'Namibia',
 'Sao Tome and Principe',
 'South Sudan',
 'South Africa',
 'Uganda',
 'Kenya',
 'Nigeria',
 'Mali',
 'Sudan',
 'Liberia',
 'Togo',
 'United Republic of Tanzania',
 'Gambia',
 'Rwanda',
 'Guinea',
 'Ghana',
 'Cameroon',
 'Burkina Faso',
 'Senegal',
 'Sierra Leone',
 'Guinea-Bissau']

In [4]:
# =============================================================================
## 2. GET DESCRIPTIVES OF DATASET
# =============================================================================
print(df.dtypes)
print(df.columns)
print(df.describe(include='all'))

## sanity check to ensure the spread of the data is correct
print('sanity check 1, ensure appropriate years and countries are represented')
list_country=list(list(df['location_name'].drop_duplicates()))
print('country list', list_country)
#inspect for NAs
print('following checks for completeness of all columns')
print('is country_id null', df['location_id'].isnull().values.any())
print('is risk_id null', df['rei_id'].isnull().values.any())
print('is risk_name null', df['rei_name'].isnull().values.any())
print('is year null', df['year'].isnull().values.any())
print('is sex null', df['sex_id'].isnull().values.any())
print('is age_group null', df['age_id'].isnull().values.any())
print('is daly_mean null', df['val'].isnull().values.any())
print('these age groups have been pulled', df['age_name'].unique())
print('these sex groups have been pulled', df['sex_name'].unique())
print('these years have been pulled', df['year'].unique())
print('these risks have been pulled', df['rei_name'].unique())

# =============================================================================
# ## 3. FILTER, RENAME AND CLEAN
# =============================================================================
#filter only the wanted columns
df = df.loc[df['metric_id'].isin([3])]
df_filtered = df.filter(['measure_name', 'location_name','location_id',
                         'sex_name', 'age_name', 'rei_id', 'cause_name',
                         'cause_id','rei_name', 'year', 'val', 'upper',
                         'lower'])
#change dtypes if neccessary
df_filtered['val'] = df_filtered['val'].astype(int)
df_filtered['upper'] = df_filtered['upper'].astype(int)
df_filtered['lower'] = df_filtered['lower'].astype(int)

#rename columns
df_filtered = df_filtered.rename(columns={'location_name': 'country',
                                          'sex_name': 'sex',
                                          'age_name':'age',
                                          'val': 'daly_mean',
                                          'upper': 'daly_upper',
                                          'lower': 'daly_lower'})

measure_id         int64
measure_name      object
location_id        int64
location_name     object
sex_id             int64
sex_name          object
age_id             int64
age_name          object
cause_id           int64
cause_name        object
rei_id             int64
rei_name          object
metric_id          int64
metric_name       object
year               int64
val              float64
upper            float64
lower            float64
dtype: object
Index(['measure_id', 'measure_name', 'location_id', 'location_name', 'sex_id',
       'sex_name', 'age_id', 'age_name', 'cause_id', 'cause_name', 'rei_id',
       'rei_name', 'metric_id', 'metric_name', 'year', 'val', 'upper',
       'lower'],
      dtype='object')
        measure_id                            measure_name   location_id  \
count      28200.0                                   28200  28200.000000   
unique         NaN                                       1           NaN   
top            NaN  DALYs (Disability-Adju

In [5]:
df_filtered.location_id.unique()

array([170, 182, 181, 172, 168, 176, 179, 178, 193, 175, 169, 213, 177,
       204, 212, 203, 184, 191, 205, 173, 197, 198, 200, 171, 187, 194,
       195, 215, 435, 196, 190, 180, 214, 211, 522, 210, 218, 189, 206,
       185, 208, 207, 202, 201, 216, 217, 209])

In [6]:
#fix gender naming
#fix naming of the gender values
df_filtered = df_filtered.replace({'Both': 'btsx','Male':'mle','Female':'fmle'})

In [7]:
#only filter our level 3 WASH risk factors
df_WASH =df_filtered.copy()

In [8]:
df_WASH

,measure_name,country,location_id,sex,age,rei_id,cause_name,cause_id,rei_name,year,daly_mean,daly_upper,daly_lower
0,DALYs (Disability-Adjusted Life Years),Congo,170,btsx,All ages,82,Respiratory infections and tuberculosis,956,"Unsafe water, sanitation, and handwashing",1990,1094,1838,505
1,DALYs (Disability-Adjusted Life Years),Congo,170,btsx,All ages,238,Respiratory infections and tuberculosis,956,No access to handwashing facility,1990,1094,1838,505
2,DALYs (Disability-Adjusted Life Years),Malawi,182,btsx,All ages,82,Respiratory infections and tuberculosis,956,"Unsafe water, sanitation, and handwashing",1990,2577,4112,1112
3,DALYs (Disability-Adjusted Life Years),Malawi,182,btsx,All ages,238,Respiratory infections and tuberculosis,956,No access to handwashing facility,1990,2577,4112,1112
4,DALYs (Disability-Adjusted Life Years),Malawi,182,btsx,All ages,82,Enteric infections,957,"Unsafe water, sanitation, and handwashing",1990,17485,22842,12345
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28195,DALYs (Disability-Adjusted Life Years),Sao Tome and Principe,215,btsx,All ages,83,"Communicable, maternal, neonatal, and nutritio...",295,Unsafe water source,2019,356,509,224
28196,DALYs (Disability-Adjusted Life Years),Sao Tome and Principe,215,btsx,All ages,84,"Communicable, maternal, neonatal, and nutritio...",295,Unsafe sanitation,2019,203,280,140
28197,DALYs (Disability-Adjusted Life Years),Sao Tome and Principe,215,btsx,All ages,238,"Communicable, maternal, neonatal, and nutritio...",295,No access to handwashing facility,2019,438,645,256
28198,DALYs (Disability-Adjusted Life Years),Sao Tome and Principe,215,btsx,All ages,82,Lower respiratory infections,322,"Unsafe water, sanitation, and handwashing",2019,319,511,139


In [9]:
print(df_WASH.rei_id.unique())
print(df_WASH.rei_name.unique())
df_WASH = df_WASH.loc[df_WASH['rei_id'].isin([238,83,84])]
print(df_WASH)

[ 82 238  83  84]
['Unsafe water, sanitation, and handwashing'
 'No access to handwashing facility' 'Unsafe water source'
 'Unsafe sanitation']
                                 measure_name                country  \
1      DALYs (Disability-Adjusted Life Years)                  Congo   
3      DALYs (Disability-Adjusted Life Years)                 Malawi   
5      DALYs (Disability-Adjusted Life Years)                 Malawi   
6      DALYs (Disability-Adjusted Life Years)                 Malawi   
7      DALYs (Disability-Adjusted Life Years)                 Malawi   
...                                       ...                    ...   
28193  DALYs (Disability-Adjusted Life Years)  Sao Tome and Principe   
28195  DALYs (Disability-Adjusted Life Years)  Sao Tome and Principe   
28196  DALYs (Disability-Adjusted Life Years)  Sao Tome and Principe   
28197  DALYs (Disability-Adjusted Life Years)  Sao Tome and Principe   
28199  DALYs (Disability-Adjusted Life Years)  Sao Tome and Prin

In [10]:

#get iso3 codes for countries
##mport the function 
import pycountry
def findCountryAlpha3 (country_name):
    """Producing current iso alpha 3 codes from standardized country names"""
    try:
        return pycountry.countries.get(name=country_name).alpha_3
    except:
        return ('')

In [17]:
##create iso
df_WASH['iso'] = df_WASH.apply(lambda row: findCountryAlpha3(row.country), axis=1)

/var/folders/3l/pgxm2f9j3bbgbjcj2pyxl__00000gp/T/ipykernel_2958/2553773447.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_WASH['iso'] = df_WASH.apply(lambda row: findCountryAlpha3(row.country), axis=1)


In [18]:
df_WASH

,measure_name,country,location_id,sex,age,rei_id,cause_name,cause_id,rei_name,year,daly_mean,daly_upper,daly_lower,iso
1,DALYs (Disability-Adjusted Life Years),Congo,170,btsx,All ages,238,Respiratory infections and tuberculosis,956,No access to handwashing facility,1990,1094,1838,505,COG
3,DALYs (Disability-Adjusted Life Years),Malawi,182,btsx,All ages,238,Respiratory infections and tuberculosis,956,No access to handwashing facility,1990,2577,4112,1112,MWI
5,DALYs (Disability-Adjusted Life Years),Malawi,182,btsx,All ages,83,Enteric infections,957,Unsafe water source,1990,15669,20898,10629,MWI
6,DALYs (Disability-Adjusted Life Years),Malawi,182,btsx,All ages,84,Enteric infections,957,Unsafe sanitation,1990,11806,15408,8333,MWI
7,DALYs (Disability-Adjusted Life Years),Malawi,182,btsx,All ages,238,Enteric infections,957,No access to handwashing facility,1990,5867,8069,3898,MWI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28193,DALYs (Disability-Adjusted Life Years),Sao Tome and Principe,215,btsx,All ages,238,All causes,294,No access to handwashing facility,2019,438,645,256,STP
28195,DALYs (Disability-Adjusted Life Years),Sao Tome and Principe,215,btsx,All ages,83,"Communicable, maternal, neonatal, and nutritio...",295,Unsafe water source,2019,356,509,224,STP
28196,DALYs (Disability-Adjusted Life Years),Sao Tome and Principe,215,btsx,All ages,84,"Communicable, maternal, neonatal, and nutritio...",295,Unsafe sanitation,2019,203,280,140,STP
28197,DALYs (Disability-Adjusted Life Years),Sao Tome and Principe,215,btsx,All ages,238,"Communicable, maternal, neonatal, and nutritio...",295,No access to handwashing facility,2019,438,645,256,STP


In [19]:
##check the uniqueness and coverage of the iso codes
inspect = df_WASH[df_WASH['iso'].isna()]
print(inspect['iso'].unique())

[]


In [20]:
list_iso = list(df_WASH.iso)
list_iso = list( dict.fromkeys(list_iso) )
print(list_iso)

['COG', 'MWI', 'MDG', 'GNQ', 'AGO', 'COM', 'ETH', 'ERI', 'BWA', 'BDI', 'CAF', 'NER', 'DJI', 'TCD', 'MRT', 'CPV', 'MOZ', 'ZMB', 'CIV', 'GAB', 'SWZ', 'ZWE', 'BEN', '', 'SOM', 'LSO', 'NAM', 'STP', 'SSD', 'ZAF', 'UGA', 'KEN', 'NGA', 'MLI', 'SDN', 'LBR', 'TGO', 'GMB', 'RWA', 'GIN', 'GHA', 'CMR', 'BFA', 'SEN', 'SLE', 'GNB']


In [15]:
df_WASH

,measure_name,country,location_id,sex,age,rei_id,cause_name,cause_id,rei_name,year,daly_mean,daly_upper,daly_lower,iso
1,DALYs (Disability-Adjusted Life Years),Congo,170,btsx,All ages,238,Respiratory infections and tuberculosis,956,No access to handwashing facility,1990,1094,1838,505,COG
3,DALYs (Disability-Adjusted Life Years),Malawi,182,btsx,All ages,238,Respiratory infections and tuberculosis,956,No access to handwashing facility,1990,2577,4112,1112,MWI
5,DALYs (Disability-Adjusted Life Years),Malawi,182,btsx,All ages,83,Enteric infections,957,Unsafe water source,1990,15669,20898,10629,MWI
6,DALYs (Disability-Adjusted Life Years),Malawi,182,btsx,All ages,84,Enteric infections,957,Unsafe sanitation,1990,11806,15408,8333,MWI
7,DALYs (Disability-Adjusted Life Years),Malawi,182,btsx,All ages,238,Enteric infections,957,No access to handwashing facility,1990,5867,8069,3898,MWI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28193,DALYs (Disability-Adjusted Life Years),Sao Tome and Principe,215,btsx,All ages,238,All causes,294,No access to handwashing facility,2019,438,645,256,STP
28195,DALYs (Disability-Adjusted Life Years),Sao Tome and Principe,215,btsx,All ages,83,"Communicable, maternal, neonatal, and nutritio...",295,Unsafe water source,2019,356,509,224,STP
28196,DALYs (Disability-Adjusted Life Years),Sao Tome and Principe,215,btsx,All ages,84,"Communicable, maternal, neonatal, and nutritio...",295,Unsafe sanitation,2019,203,280,140,STP
28197,DALYs (Disability-Adjusted Life Years),Sao Tome and Principe,215,btsx,All ages,238,"Communicable, maternal, neonatal, and nutritio...",295,No access to handwashing facility,2019,438,645,256,STP


In [22]:
##set iso manually for countries that were missed
df_WASH.loc[df_WASH['country'] == 'United Republic of Tanzania', 'iso'] = 'TZA'
df_WASH.loc[df_WASH['country'] == 'Democratic Republic of the Congo', 'iso'] = 'COD'

In [23]:
df_WASH

,measure_name,country,location_id,sex,age,rei_id,cause_name,cause_id,rei_name,year,daly_mean,daly_upper,daly_lower,iso
1,DALYs (Disability-Adjusted Life Years),Congo,170,btsx,All ages,238,Respiratory infections and tuberculosis,956,No access to handwashing facility,1990,1094,1838,505,COG
3,DALYs (Disability-Adjusted Life Years),Malawi,182,btsx,All ages,238,Respiratory infections and tuberculosis,956,No access to handwashing facility,1990,2577,4112,1112,MWI
5,DALYs (Disability-Adjusted Life Years),Malawi,182,btsx,All ages,83,Enteric infections,957,Unsafe water source,1990,15669,20898,10629,MWI
6,DALYs (Disability-Adjusted Life Years),Malawi,182,btsx,All ages,84,Enteric infections,957,Unsafe sanitation,1990,11806,15408,8333,MWI
7,DALYs (Disability-Adjusted Life Years),Malawi,182,btsx,All ages,238,Enteric infections,957,No access to handwashing facility,1990,5867,8069,3898,MWI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28193,DALYs (Disability-Adjusted Life Years),Sao Tome and Principe,215,btsx,All ages,238,All causes,294,No access to handwashing facility,2019,438,645,256,STP
28195,DALYs (Disability-Adjusted Life Years),Sao Tome and Principe,215,btsx,All ages,83,"Communicable, maternal, neonatal, and nutritio...",295,Unsafe water source,2019,356,509,224,STP
28196,DALYs (Disability-Adjusted Life Years),Sao Tome and Principe,215,btsx,All ages,84,"Communicable, maternal, neonatal, and nutritio...",295,Unsafe sanitation,2019,203,280,140,STP
28197,DALYs (Disability-Adjusted Life Years),Sao Tome and Principe,215,btsx,All ages,238,"Communicable, maternal, neonatal, and nutritio...",295,No access to handwashing facility,2019,438,645,256,STP


In [27]:
df_WASH.query('location_id == 171')

,measure_name,country,location_id,sex,age,rei_id,cause_name,cause_id,rei_name,year,daly_mean,daly_upper,daly_lower,iso
144,DALYs (Disability-Adjusted Life Years),Democratic Republic of the Congo,171,btsx,All ages,83,Diarrheal diseases,302,Unsafe water source,1990,6136,8764,4128,COD
145,DALYs (Disability-Adjusted Life Years),Democratic Republic of the Congo,171,btsx,All ages,84,Diarrheal diseases,302,Unsafe sanitation,1990,4565,6368,3209,COD
146,DALYs (Disability-Adjusted Life Years),Democratic Republic of the Congo,171,btsx,All ages,238,Diarrheal diseases,302,No access to handwashing facility,1990,2358,3345,1591,COD
192,DALYs (Disability-Adjusted Life Years),Democratic Republic of the Congo,171,btsx,All ages,238,Lower respiratory infections,322,No access to handwashing facility,1990,2503,3979,1176,COD
279,DALYs (Disability-Adjusted Life Years),Democratic Republic of the Congo,171,btsx,All ages,238,Respiratory infections and tuberculosis,956,No access to handwashing facility,1991,2449,3873,1165,COD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27946,DALYs (Disability-Adjusted Life Years),Democratic Republic of the Congo,171,btsx,All ages,83,Diarrheal diseases,302,Unsafe water source,2019,2326,4305,1021,COD
27947,DALYs (Disability-Adjusted Life Years),Democratic Republic of the Congo,171,btsx,All ages,84,Diarrheal diseases,302,Unsafe sanitation,2019,1648,3039,739,COD
27948,DALYs (Disability-Adjusted Life Years),Democratic Republic of the Congo,171,btsx,All ages,238,Diarrheal diseases,302,No access to handwashing facility,2019,899,1687,389,COD
28061,DALYs (Disability-Adjusted Life Years),Democratic Republic of the Congo,171,btsx,All ages,238,Lower respiratory infections,322,No access to handwashing facility,2019,570,943,259,COD


In [32]:
list(df_WASH.iso.unique())
df_WASH.head()
df_WASH.columns

Index(['measure_name', 'country', 'location_id', 'sex', 'age', 'rei_id',
       'cause_name', 'cause_id', 'rei_name', 'year', 'daly_mean', 'daly_upper',
       'daly_lower', 'iso'],
      dtype='object')

In [24]:
df_WASH.iso.unique()

array(['COG', 'MWI', 'MDG', 'GNQ', 'AGO', 'COM', 'ETH', 'ERI', 'BWA',
       'BDI', 'CAF', 'NER', 'DJI', 'TCD', 'MRT', 'CPV', 'MOZ', 'ZMB',
       'CIV', 'GAB', 'SWZ', 'ZWE', 'BEN', 'COD', 'SOM', 'LSO', 'NAM',
       'STP', 'SSD', 'ZAF', 'UGA', 'KEN', 'NGA', 'MLI', 'SDN', 'LBR',
       'TGO', 'TZA', 'GMB', 'RWA', 'GIN', 'GHA', 'CMR', 'BFA', 'SEN',
       'SLE', 'GNB'], dtype=object)

In [28]:
#safe data to csv as back up
os.getcwd()
df_WASH.to_csv('df_DALY_SSA.csv')